# Assignment 6

## Group members

- Erdem Halil - gushaliler@student.gu.se

## Problem 1

In [ ]:
import math

class Graph:
    def __init__(self, gdict={}):
        self.gdict = gdict
        self.colour = {}
        self.distance = {}
        self.predecessor = {}
        self.finish = {}

    def get_vertices(self):
        return list(self.gdict.keys())

    def get_edges(self):
        edges = []
        for vertex in self.gdict:
            for next_vertex in self.gdict[vertex]:
                # use tuple instead of set
                if (vertex, next_vertex) not in edges:
                    edges.append((vertex, next_vertex))
        return edges

    def print_path(self, s, v):
        if v in self.gdict.keys():
            if v == s:
                print(s)
            elif self.predecessor[v] == None:
                print("There is no path from", s, "to", v, "exists.")
            else:
                self.print_path(s, self.predecessor[v])
                print(v)
        else:
            print("Node with key", v, "is not in the graph.")

    # initialise_single_source runs in O(V)
    def initialise_single_source(self, s):
        for v in self.get_vertices():
            self.distance[v] = math.inf
            self.predecessor[v] = None
        self.distance[s] = 0

    def get_weight(self, u, v):
        return self.gdict[u][v]

    def relax(self, u, v):
        if self.distance[v] > self.distance[u] + self.get_weight(u, v):
            self.distance[v] = self.distance[u] + self.get_weight(u, v)
            self.predecessor[v] = u

    def shortest_path_dijkstra(self, s, v):
        print("TODO")

    def shortest_path_dag(self, s, v):
        print("TODO")
    
    

In [ ]:
# Input graphs
adjacency = {
    "r": {"s": 5, "t": 3},
    "s": {"x": 6, "t": 2},
    "t": {"x": 7, "y": 4, "z": 2},
    "x": {"y": -1, "z": 1},
    "y": {"z": -2},
    "z": {}
}

adjacency2 = {
    "s": {"t": 5, "y": 10},
    "t": {"x": 1, "y": 2},
    "x": {"z": 4},
    "y": {"x": 9,"t": 3, "z": 2},
    "z": {"s": 7,"x": 6}
}

graph = Graph(adjacency2)
graph2 = Graph(adjacency)
graph.shortest_path_dijkstra("s", "z") # This is just a method to print out the output (total path).
graph2.shortest_path_dag("s", "z") # This is just a method to print out the output (total path).